In [15]:
import cv2 as cv
import numpy as np

In [16]:
def avg_frame(video_path, max_frames=300, step=3):
    cap = cv.VideoCapture(video_path)
    count = 0
    avg = None
    frame_index = 0

    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index % step != 0:
            frame_index += 1
            continue

        frame_index += 1
        frame_f = frame.astype(np.float32)

        if avg is None:
            avg = frame_f
        else:
            avg += frame_f

        count += 1

    cap.release()

    if count == 0:
        raise Exception(f"ERROR: No frames loaded from video: {video_path}")

    return (avg / count).astype(np.uint8)


In [17]:
def detect_cars(frame, background_blur, mask_roi):
    """
    Recebe um frame e devolve bounding boxes detectados dentro da ROI.
    """
    # pré-processamento
    frame_blur = cv.GaussianBlur(frame, (21, 21), 0)
    diff = cv.absdiff(frame_blur, background_blur)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)

    masked = cv.bitwise_and(gray, mask_roi)

    # threshold automático
    _, thresh = cv.threshold(masked, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # morfologia
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))
    cleaned = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
    cleaned = cv.morphologyEx(cleaned, cv.MORPH_OPEN, kernel, iterations=1)

    contours, _ = cv.findContours(cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    boxes = []

    for cnt in contours:
        area = cv.contourArea(cnt)
        if area < 800:
            continue

        x, y, w, h = cv.boundingRect(cnt)

        # filtrar sombras (objetos muito estreitos/altos)
        if w / float(h) < 0.3:
            continue

        boxes.append((x, y, w, h))

    return boxes


In [ ]:
def count_cars(video_path, output_path, roi_polygon):
    # background
    background = avg_frame(video_path)
    background_blur = cv.GaussianBlur(background, (21, 21), 0)

    cap = cv.VideoCapture(video_path)
    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv.CAP_PROP_FPS)

    # máscara ROI
    mask_roi = np.zeros((h, w), dtype=np.uint8)
    cv.fillPoly(mask_roi, [roi_polygon], 255)

    # writer
    out = cv.VideoWriter(output_path, cv.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # tracking simples
    prev_centroids = {}
    next_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # DETEÇÃO
        boxes = detect_cars(frame, background_blur, mask_roi)

        new_centroids = {}

        for (x, y, w2, h2) in boxes:
            cx = x + w2 // 2
            cy = y + h2 // 2

            # associar centroid ao ID (match simples por proximidade)
            assigned_id = None
            for obj_id, (px, py) in prev_centroids.items():
                if abs(cx - px) < 30 and abs(cy - py) < 40:
                    assigned_id = obj_id
                    break

            if assigned_id is None:
                assigned_id = next_id
                next_id += 1

            new_centroids[assigned_id] = (cx, cy)

            # desenhar bounding box
            cv.rectangle(frame, (x, y), (x+w2, y+h2), (0, 255, 0), 2)
            cv.putText(frame, f"ID:{assigned_id}", (x, y-5),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

        prev_centroids = new_centroids

        # desenhar a ROI
        cv.polylines(frame, [roi_polygon], True, (255, 0, 0), 2)

        out.write(frame)
        cv.imshow("Car Counting", frame)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv.destroyAllWindows()

    print("Done! Cars counted:", assigned_id)


In [ ]:
video_path = "resources/AutoEstrada.avi"

roi_polygon = np.array([
    [100, 70],
    [155, 70],
    [230, 240],
    [5, 240]
], dtype=np.int32)

count_cars(video_path, "cars_counted.mp4", roi_polygon)


Done! Cars counted: 12
